# Phase 6: Maia2 Humanness Analysis

Analyze moves for human-like patterns using Maia2 neural network predictions.

**Inputs:**
- Phase 1: games.parquet, raw games
- Phase 3: high_priority_games.json

**Outputs:**
- `maia2_analysis.parquet` - Humanness scores per game
- `maia2_positions.parquet` - Position-level Maia2 data (probability, rank) for ALL positions
- `surprising_moves.json` - Moves that deviate from human predictions

In [ ]:
# Parameters (injected by Papermill)
username = "default_user"  # Chess.com username
maia_model_path = ""  # Path to Maia2 model (empty string = auto-detect)

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')
from common import (
    setup_notebook, validate_parameters, print_section, print_subsection,
    get_user_data_dir, save_phase_output, load_phase_output,
    load_dataset_parquet, load_cached_games_v2,
    analyze_game_maia2, calculate_humanness_score, get_surprising_moves,
    compare_to_stockfish,
    PROJECT_ROOT,
    detect_game_phase, GamePhase,  # For filtering opening phase moves
)
import chess
import chess.pgn
import io
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path

setup_notebook()
validate_parameters(username)

In [ ]:
# Check for Maia2 library availability
print_section(f"MAIA2 HUMANNESS ANALYSIS: {username}")

# Check if maia2 library is installed
try:
    import maia2
    maia_available = True
    print("Maia2 library found")
except ImportError:
    maia_available = False
    print("WARNING: maia2 library not installed. Humanness analysis will be limited.")
    print("Install with: pip install maia2")
    print("Models will be downloaded automatically on first use.")

In [ ]:
# Load data
user_data_dir = get_user_data_dir(username)
MIN_GAMES_FOR_ANALYSIS = 10  # Minimum games to analyze

# Load high priority games from phase 3, or fall back to selection
try:
    priority_data = load_phase_output(username, "phase3", "high_priority_games.json")
    game_ids_to_analyze = set(priority_data['game_ids'])
    print(f"High priority games from phase 3: {len(game_ids_to_analyze)}")
except FileNotFoundError:
    print("Phase 3 output not found, will select games for analysis...")
    game_ids_to_analyze = set()

# Load raw games
all_games_raw, _ = load_cached_games_v2(user_data_dir)
print(f"Raw games loaded: {len(all_games_raw)}")

# Ensure we have enough games for meaningful analysis
if len(game_ids_to_analyze) < MIN_GAMES_FOR_ANALYSIS:
    print(f"\nOnly {len(game_ids_to_analyze)} games from phase 3, need at least {MIN_GAMES_FOR_ANALYSIS}")
    print("Selecting additional games for analysis...")
    
    # Load games dataframe to get game IDs
    games_df = pd.DataFrame(load_dataset_parquet(user_data_dir / "games.parquet"))
    all_game_ids = set(games_df['game_id'].tolist())
    
    # Add games until we have enough (prioritize wins)
    wins = games_df[games_df['player_result'] == 'win']['game_id'].tolist()
    others = games_df[games_df['player_result'] != 'win']['game_id'].tolist()
    
    needed = min(MIN_GAMES_FOR_ANALYSIS, len(all_game_ids)) - len(game_ids_to_analyze)
    
    for game_id in wins:
        if game_id not in game_ids_to_analyze:
            game_ids_to_analyze.add(game_id)
            needed -= 1
            if needed <= 0:
                break
    
    if needed > 0:
        for game_id in others:
            if game_id not in game_ids_to_analyze:
                game_ids_to_analyze.add(game_id)
                needed -= 1
                if needed <= 0:
                    break
    
    print(f"Final game count for analysis: {len(game_ids_to_analyze)}")

In [ ]:
# Helper to extract positions and metadata from PGN
def extract_game_data(game_id: str, all_games: list, target_username: str):
    """Extract positions, player color, and ratings from a game."""
    for game in all_games:
        url = game.get('url', '')
        if game_id in url or url.endswith(f'/{game_id}'):
            pgn_str = game.get('pgn', '')
            if not pgn_str:
                return None
            
            pgn = chess.pgn.read_game(io.StringIO(pgn_str))
            if not pgn:
                return None
            
            # Determine player color and get ratings
            white_player = pgn.headers.get('White', '').lower()
            black_player = pgn.headers.get('Black', '').lower()
            player_is_white = target_username.lower() == white_player
            
            # Get Elo ratings from headers
            try:
                white_elo = int(pgn.headers.get('WhiteElo', 1500))
                black_elo = int(pgn.headers.get('BlackElo', 1500))
            except ValueError:
                white_elo = black_elo = 1500
            
            if player_is_white:
                player_elo = white_elo
                opponent_elo = black_elo
            else:
                player_elo = black_elo
                opponent_elo = white_elo
            
            # Determine game type from time control
            time_control = pgn.headers.get('TimeControl', '')
            if 'rapid' in game.get('time_class', '').lower() or (time_control and int(time_control.split('+')[0]) >= 600):
                game_type = 'rapid'
            else:
                game_type = 'blitz'
            
            # Extract positions with player moves
            positions = []
            board = pgn.board()
            for ply, move in enumerate(pgn.mainline_moves()):
                is_player_move = (ply % 2 == 0) == player_is_white
                if is_player_move:
                    positions.append({
                        'fen': board.fen(),
                        'move': move.uci(),
                        'ply': ply,
                    })
                board.push(move)
            
            return {
                'pgn': pgn,
                'positions': positions,
                'player_is_white': player_is_white,
                'player_elo': player_elo,
                'opponent_elo': opponent_elo,
                'game_type': game_type,
            }
    return None

# Extract data for games to analyze
games_to_process = []
for game_id in game_ids_to_analyze:
    game_data = extract_game_data(game_id, all_games_raw, username)
    if game_data and game_data['positions']:
        games_to_process.append((game_id, game_data))

print(f"Games with valid positions: {len(games_to_process)}")

In [ ]:
# Run Maia2 analysis (if available) or simplified humanness analysis
print_subsection("ANALYZING HUMANNESS")

# Threshold for flagging surprising moves - only flag moves with <1% probability
# Moves above this threshold are generally within normal human variation
SURPRISING_MOVE_THRESHOLD = 0.01  # 1%
# Disable rank-based filtering by setting a very high threshold
SURPRISING_RANK_THRESHOLD = 100  # Effectively disabled

def is_opening_position(fen: str) -> bool:
    """Check if a position is still in the opening phase."""
    try:
        board = chess.Board(fen)
        phase = detect_game_phase(board)
        return phase == GamePhase.OPENING
    except Exception:
        return False

maia_results = []
all_surprising_moves = []
all_position_data = []  # Store ALL position-level Maia2 data for cross-reference

for game_id, game_data in tqdm(games_to_process, desc="Analyzing games"):
    try:
        if maia_available:
            # Full Maia2 analysis using correct API
            maia2_result = analyze_game_maia2(
                positions=game_data['positions'],
                player_elo=game_data['player_elo'],
                opponent_elo=game_data['opponent_elo'],
                game_type=game_data['game_type'],
            )
            
            if maia2_result:
                # calculate_humanness_score expects list of Maia2Result objects
                humanness = calculate_humanness_score(maia2_result)
                surprising = get_surprising_moves(
                    maia2_result, 
                    probability_threshold=SURPRISING_MOVE_THRESHOLD,
                    rank_threshold=SURPRISING_RANK_THRESHOLD,
                )
                
                maia_results.append({
                    'game_id': game_id,
                    'humanness_score': humanness.get('humanness_score', 0),
                    'moves_analyzed': humanness.get('num_positions', 0),
                    'avg_maia_probability': humanness.get('avg_move_probability', 0),
                    'top_choice_rate': humanness.get('top_choice_rate', 0),
                    'avg_move_rank': humanness.get('avg_move_rank', 0),
                    'surprising_moves': len(surprising),
                })
                
                # Save ALL position-level Maia2 data for cross-reference with engine analysis
                for r in maia2_result:
                    all_position_data.append({
                        'game_id': game_id,
                        'fen': r.fen,
                        'move': r.played_move,
                        'probability': r.move_probability,
                        'rank': r.move_rank,
                        'top_move': r.top_move,
                        'top_move_probability': r.top_move_probability,
                    })
                
                if surprising:
                    # Filter out opening phase moves - opening moves are expected to be "book"
                    # moves and shouldn't be flagged as surprising
                    filtered_moves = []
                    for s in surprising:
                        if not is_opening_position(s.fen):
                            filtered_moves.append({
                                'fen': s.fen, 
                                'move': s.played_move, 
                                'probability': s.move_probability, 
                                'rank': s.move_rank,
                                'top_move': s.top_move,
                                'top_move_probability': s.top_move_probability,
                            })
                    
                    if filtered_moves:
                        all_surprising_moves.append({
                            'game_id': game_id,
                            'moves': filtered_moves,
                            'player_is_white': game_data['player_is_white'],  # For board orientation
                        })
        else:
            # Simplified analysis without Maia2 library
            # Use move complexity/commonality as proxy for humanness
            positions = game_data['positions']
            pgn = game_data['pgn']
            player_is_white = game_data['player_is_white']
            
            board = pgn.board()
            player_moves = []
            
            for ply, move in enumerate(pgn.mainline_moves()):
                is_player_move = (ply % 2 == 0) == player_is_white
                if is_player_move:
                    # Simple heuristics for "humanness"
                    legal_moves = list(board.legal_moves)
                    player_moves.append({
                        'ply': ply,
                        'move': move.uci(),
                        'legal_moves': len(legal_moves),
                        'is_capture': board.is_capture(move),
                        'is_check': board.gives_check(move),
                    })
                board.push(move)
            
            if player_moves:
                # Calculate simplified humanness score
                # (captures and checks are more "obviously human")
                obvious_moves = sum(1 for m in player_moves if m['is_capture'] or m['is_check'])
                humanness = obvious_moves / len(player_moves) if player_moves else 0
                
                maia_results.append({
                    'game_id': game_id,
                    'humanness_score': humanness,
                    'moves_analyzed': len(player_moves),
                    'avg_maia_probability': 0,  # Not available without maia2
                    'top_choice_rate': 0,
                    'avg_move_rank': 0,
                    'surprising_moves': 0,
                    'note': 'simplified_analysis'
                })
    except Exception as e:
        print(f"Error analyzing {game_id}: {e}")
        continue

print(f"\nAnalyzed {len(maia_results)} games")
print(f"Games with surprising (non-opening) moves: {len(all_surprising_moves)}")
print(f"Total positions with Maia2 data: {len(all_position_data)}")

In [ ]:
# Display results
print_subsection("HUMANNESS RESULTS")

if maia_results:
    maia_df = pd.DataFrame(maia_results)
    
    print(f"Games analyzed: {len(maia_df)}")
    print(f"\nHumanness statistics:")
    print(f"  Average humanness score: {maia_df['humanness_score'].mean():.2f}")
    
    if maia_available:
        print(f"  Average Maia probability: {maia_df['avg_maia_probability'].mean():.2%}")
        print(f"  Total surprising moves: {maia_df['surprising_moves'].sum()}")
    
    # Games with lowest humanness (most engine-like)
    print(f"\nGames with lowest humanness:")
    print(maia_df.nsmallest(5, 'humanness_score')[['game_id', 'humanness_score', 'moves_analyzed']].to_string())
else:
    maia_df = pd.DataFrame()
    print("No Maia2 results available.")

In [ ]:
# Save outputs
if not maia_df.empty:
    save_phase_output(username, "phase6", "maia2_analysis.parquet", maia_df)

# Save ALL position-level Maia2 data for cross-reference with other phases
if all_position_data:
    position_df = pd.DataFrame(all_position_data)
    save_phase_output(username, "phase6", "maia2_positions.parquet", position_df)
    print(f"Position-level Maia2 data saved: {len(position_df)} positions")

# Save surprising moves
surprising_output = {
    "username": username,
    "maia_available": maia_available,
    "games_analyzed": len(maia_df) if not maia_df.empty else 0,
    "avg_humanness": float(maia_df['humanness_score'].mean()) if not maia_df.empty else 0,
    "surprising_moves": all_surprising_moves,
}
save_phase_output(username, "phase6", "surprising_moves.json", surprising_output)

print(f"\nPhase 6 complete!")

In [ ]:
# Visualization
import matplotlib.pyplot as plt

if not maia_df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Humanness score distribution
    axes[0].hist(maia_df['humanness_score'], bins=20, color='steelblue', edgecolor='white')
    axes[0].axvline(maia_df['humanness_score'].mean(), color='red', linestyle='--', 
                   label=f"Mean: {maia_df['humanness_score'].mean():.2f}")
    axes[0].set_xlabel('Humanness Score')
    axes[0].set_ylabel('Games')
    axes[0].set_title('Humanness Score Distribution')
    axes[0].legend()
    
    # Humanness vs moves analyzed
    axes[1].scatter(maia_df['moves_analyzed'], maia_df['humanness_score'], alpha=0.6)
    axes[1].set_xlabel('Moves Analyzed')
    axes[1].set_ylabel('Humanness Score')
    axes[1].set_title('Humanness vs Game Length')
    
    plt.tight_layout()
    plt.show()